# Appendix

## Group 5
## L. Terlinden-Ruhl (5863937),  A. Magherini (5838215)
Site: USGS 09180500 COLORADO RIVER NEAR CISCO, UT

## 1 - River Selection and Data Collection

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sci
import pathlib
import datetime

%matplotlib inline

In [4]:
# Create path to the data folder

data_dir = pathlib.Path('/work').expanduser()
site = list(sorted(data_dir.glob('Colorado river.txt')))
site[0]

In [5]:
df = pd.read_csv(site[0], skiprows = 49, header = 0, 
                     sep='\t', engine='python').drop(['agency_cd', 'site_no'], axis = 1).drop([0], axis = 0)

IndexError: list index out of range

In [ ]:
# Drop columns that will not be used

df.drop(df.columns[12:], axis =1, inplace = True)
df.drop(df.columns[1:3], axis = 1, inplace = True)
df.drop(df.columns[1:3], axis = 1, inplace = True)
df.drop(df.columns[2:7], axis = 1, inplace = True)

In [ ]:
# sediment discharge stops recording in 1984, discharge until 2022

# Rename columns, for better reference

df.rename(columns = {'datetime': 'Date',
                     '142741_00060_00003': r'Discharge $(m^3 s^{-1})$',
                     '142745_80155_00003': r'Suspended sediment discharge $(ton$ $day^{-1})$'
                     }, inplace = True)

# store column names making it easier to call
col = df.columns.values

In [ ]:
# change dtype of columns to manipulate data more easily
df[col[1]] = df[col[1]].astype('int') * 0.0283168
df[col[2]] = df[col[2]].astype('float') * 0.91
df['Date'] = pd.to_datetime(df['Date'])
df

In [ ]:
# Easy plots to understand the data (question 1)
df.plot(kind = 'scatter', x = 'Date',
        y = col[1], title = 'Timeseries of Discharge', color='b');    
plt.show()
df.plot(kind = 'scatter', x = 'Date',
        y = col[2],
        title = 'Timeseries of Sediment Discharge', color='b');    
plt.show()
df.plot(kind = 'scatter', x = col[1],
        y = col[2], title = 'Bivariate analysis of complete dataset', color='b');    
plt.show()
df.hist(col[1], density = True, color='b')
plt.ylabel('Density, [-]')
plt.show()
df.hist(col[2], density = True, color='b')
plt.ylabel('Density, [-]')
plt.show()

In [ ]:
def yearly_maxima(data, column):
    '''
    Function that calculates the yearly maximum of parameters in data frame for the requested column index
    '''
    col = data.columns.values
    idx_max = data.groupby(pd.DatetimeIndex(data[col[0]]).year)[col[column]].idxmax()
    max_list = data.loc[idx_max]
    return max_list

In [ ]:
yearly_maxima_discharge = yearly_maxima(df, 1) # maxima of discharge, accompanying values of sediment
yearly_maxima_discharge

In [ ]:
# Plot flow discharge historic data with maxima
df.plot(kind = 'scatter', x= 'Date', y = col[1], color='b')
plt.scatter(yearly_maxima_discharge['Date'], yearly_maxima_discharge[col[1]],
            40, 'r', label='yearly maximum')
plt.title(col[1])
plt.legend(loc='best');

## 2 - Distribution fitting

In [ ]:
df.describe()

In [ ]:
#annual maxima statistics of flow discharge and related sediment discharge

yearly_maxima_discharge.describe()

## Fit a GEV distribution to annual maxima

In [ ]:
def calculate_ecdf(extreme_value_data, column):
    """
    Fit an emperical cumulative distribution function to known values for a given column index 
    """
    col = extreme_value_data.columns.values          
    # Sort the dataframe from high to low values based on the column of the extreme value     
    sorted_data = extreme_value_data.sort_values(by = col[column], ascending=True)

    sorted_data = sorted_data[2:] # removing the first entry gives a better fit for genextreme

    sorted_data.reset_index(drop=True, inplace=True)     
    # Reset the index, because in this specific case we have to work with an index starting from 1 not 0     
    sorted_data.index = sorted_data.index + 1      
    # Calculating the total length of the dataframe     
    N = len(sorted_data[col[column]])       
    # Filling this new column with the F(x) score     
    sorted_data['F_x'] = sorted_data.index / (N + 1)     
    ecdf = sorted_data[['F_x', col[column]]]   
    # Return the dataframe with the added column     
    return ecdf 

In [ ]:
ecdf_blockmax = calculate_ecdf(yearly_maxima_discharge, 1)
ecdf_blockmax.head()

In [ ]:
def plot_density_functions(emperical_df, x, pdf, cdf, CI_1, CI_2, quantiles, method, curve):
    '''
    Plots pdf, cdf, return period and QQ plot for a given function. Confidence intervals are included in 
    return period graph.
    '''
    # pdf
    emperical_df.hist(col[1], density = True, label = 'Emperical Histogram', color='b')
    plt.title('PDF from' + method + 'using' + curve) 
    plt.plot(x, pdf, 'r', label = 'Fitted' + curve)
    plt.ylabel('f(x)') 
    plt.xlabel(col[1]) 
    plt.grid() 
    plt.legend()
    if curve == ' genextreme': plt.yscale('log')
    plt.show()
    # cdf
    plt.scatter(emperical_df[col[1]], emperical_df['F_x'], 40, 'k', label = 'Empirical cdf') 
    plt.title('CDF from' + method + 'using' + curve) 
    plt.plot(emperical_df[col[1]], cdf, 'r', label = 'Fitted' + curve)
    plt.ylabel('F(x)') 
    plt.xlabel(col[1]) 
    plt.grid() 
    plt.legend()
    plt.ylim([0, 1])
    plt.show()
    # inv cdf
    plt.scatter(1 / (1 - emperical_df['F_x']), emperical_df[col[1]], 40, 'k',
                label = 'Empirical inverse cdf') 
    plt.title('Return Period from' + method + 'using' + curve) 
    plt.plot(1 / (1 - cdf), emperical_df[col[1]], 'r', label = 'Fitted' + curve)
    plt.plot(1 / (1 - CI_1), emperical_df[col[1]], 'b', label = 'CI' + curve) 
    plt.plot(1 / (1 - CI_2), emperical_df[col[1]], 'b') 
    plt.xlabel('Return Period [years]') 
    plt.ylabel(col[1]) 
    plt.xscale('log')
    plt.grid() 
    plt.legend()
    plt.show()
    # QQ Plot
    empirical_quantiles = emperical_df[col[1]]

    plt.figure(figsize = (6, 6))
    plt.scatter(empirical_quantiles, quantiles, 40, 'k', label = curve)
    linear = np.array([250, 2000])
    plt.plot(linear, linear, label = 'Perfect fit')
    plt.title('QQ-plot for' + curve)
    plt.ylabel(r'Theoretical quantiles (Discharge [$m^3$ $s^{-1}$])')
    plt.xlabel(r'Empirical quantiles (Discharge [$m^3$ $s^{-1}$])')
    plt.grid()
    plt.legend()
    plt.show()

In [ ]:
# compute parameters and corresponding pdf, cdf, CI, quantiles for GEV and plot them
GEV_param_blockmax = sci.genextreme.fit(ecdf_blockmax[col[1]], method = 'mle')

spacing = np.linspace(min(ecdf_blockmax[col[1]]), max(ecdf_blockmax[col[1]]),
                      len(ecdf_blockmax[col[1]]))
GEV_pdf = sci.genextreme.pdf(spacing, GEV_param_blockmax[0],
                             GEV_param_blockmax[1], GEV_param_blockmax[2])
GEV_cdf = sci.genextreme.cdf(ecdf_blockmax[col[1]], GEV_param_blockmax[0],
                             GEV_param_blockmax[1], GEV_param_blockmax[2])
GEV_quantiles = sci.genextreme.ppf(ecdf_blockmax['F_x'], GEV_param_blockmax[0],
                                   GEV_param_blockmax[1], GEV_param_blockmax[2])

sigma = 0.05 * ecdf_blockmax[col[1]]
k95 = sci.norm.ppf(1-0.05/2)
k95_sig = k95*sigma

GEV_cdf_ci1 = sci.genextreme.cdf(ecdf_blockmax[col[1]] + k95_sig, GEV_param_blockmax[0],
                             GEV_param_blockmax[1], GEV_param_blockmax[2])
GEV_cdf_ci2 = sci.genextreme.cdf(ecdf_blockmax[col[1]] - k95_sig, GEV_param_blockmax[0],
                             GEV_param_blockmax[1], GEV_param_blockmax[2])

plot_density_functions(ecdf_blockmax, spacing, GEV_pdf, GEV_cdf, GEV_cdf_ci1, GEV_cdf_ci2, 
                       GEV_quantiles, ' block maxima ', ' GEV')

Removing the first two data points was required in order to have the GEV fit.

## Gumbel distribution

In [ ]:
# compute parameters and corresponding pdf, cdf, CI, quantiles for Gumbel and plot them

gamma = 0.5772156649   #Euler constant

def alpha_beta (data):   # coefficient
    beta = (np.sqrt(6) * np.std(data)) / (np.pi)
    alpha = np.mean(data) - gamma * beta
    return alpha, beta

def gumbel_pdf(data, x):
    '''
    Function that computes the Gumbel PDF

    Input: data = data
           x    = spacing of output
    
    Output: pdf = Gumbel PDF
    '''
    alpha, beta = alpha_beta(data)

    pdf = sci.gumbel_r.pdf(x, loc = alpha, scale = beta)
    return pdf

def gumbel_cdf(data, x):
    '''
    Function that computes the Gumbel CDF

    Input: data = data
           x    = spacing of output
    
    Output: pdf = Gumbel CDF
    '''
    alpha, beta = alpha_beta(data)

    cdf = sci.gumbel_r.cdf(x, loc = alpha, scale = beta)
    return cdf

def gumbel_ppf(data, probability):
    '''
    Function that computes the Gumbel CDF

    Input: data = data
           probability = emperical cdf of data
           x    = spacing of output
    
    Output: pdf = Gumbel CDF
    '''
    alpha, beta = alpha_beta(data)

    ppf = sci.gumbel_r.ppf(probability, loc = alpha, scale = beta)
    return ppf

gum_pdf = gumbel_pdf(ecdf_blockmax[col[1]], spacing)
gum_cdf = gumbel_cdf(ecdf_blockmax[col[1]], ecdf_blockmax[col[1]])
gum_ppf = gumbel_ppf(ecdf_blockmax[col[1]], ecdf_blockmax['F_x'])

gum_cdf_ci1 = gumbel_cdf(ecdf_blockmax[col[1]] + k95_sig, ecdf_blockmax[col[1]])
gum_cdf_ci2 = gumbel_cdf(ecdf_blockmax[col[1]] - k95_sig, ecdf_blockmax[col[1]])

In [ ]:
plot_density_functions(ecdf_blockmax, spacing, gum_pdf, gum_cdf, gum_cdf_ci1, gum_cdf_ci2,
                       gum_ppf, ' block maxima ', ' Gumbel')

## Comparison of both distributions

In [ ]:
ext_spacing = np.linspace(200, 3000)
gum_cdf_ext = gumbel_cdf(ecdf_blockmax[col[1]], ext_spacing)
GEV_cdf_ext = sci.genextreme.cdf(ext_spacing, GEV_param_blockmax[0],
                             GEV_param_blockmax[1], GEV_param_blockmax[2])

plt.scatter(1 / (1 - ecdf_blockmax['F_x']), ecdf_blockmax[col[1]], 40, 'k',
            label = 'Empirical inverse cdf')
plt.title('Comparison of Return Period from' + method + 'using GEV and Gumbel') 
plt.plot(1 / (1 - gum_cdf_ext), ext_spacing, label = 'Fitted Gumbel')
plt.plot(1 / (1 - GEV_cdf_ext), ext_spacing, label = 'Fitted GEV')
plt.xlabel('Return Period [years]') 
plt.ylabel(col[1]) 
plt.xscale('log')
plt.grid() 
plt.legend()
plt.show()

## 3 - Event evaluation

In [ ]:
# drop all rows with nan values to make the columns the same length (with respect to usuable data)
df = df.iloc[:len(df[col[2]].dropna())]
df

In [ ]:
yearly_maxima_dis = yearly_maxima(df, 1)  # maxima of discharge, accompanying values sediment
yearly_maxima_dis

In [ ]:
yearly_maxima_sed = yearly_maxima(df, 2)  # maxima of sediment, accompanying values of discharge
yearly_maxima_sed

In [ ]:
# Plot suspended sediment discharge historic data with maxima
df.plot(kind = 'scatter', x= 'Date', y = col[2], color='b')
plt.scatter(yearly_maxima_sed['Date'], yearly_maxima_sed[col[2]],
            40, 'r', label='yearly maximum')
plt.title(col[2])
plt.legend(loc='best');

In [ ]:
#annual maxima statistics of sediment discharge and related flow discharge 

## this results are different from the previous ones!

yearly_maxima_sed.describe()

In [ ]:
def calculate_covariance(X1, X2):
    '''
    Computes covariance for 2 given variables
    '''
    mean_x1 = np.mean(X1)
    mean_x2 = np.mean(X2)
    diff_x1 = [item-mean_x1 for item in X1]
    diff_x2 = [item-mean_x2 for item in X2]
    product = [a*b for a,b in zip(diff_x1,diff_x2)]
    covariance = np.mean(product)
    return covariance

def pearson_correlation(X1, X2):
    '''
    Computes correlation coefficent for 2 given variables
    '''
    covariance = calculate_covariance(X1, X2)
    correl_coeff = covariance/(np.std(X1)*np.std(X2))
    return correl_coeff

In [ ]:
df.plot(kind = 'scatter', x = col[1],
        y = col[2], title = 'Bivariate analysis of complete dataset', c = 'b');
correl_coeff = pearson_correlation(df[col[1]], df[col[2]])
plt.text(1500, 800000, f'rho={correl_coeff:.3f}')   
plt.show()

In [ ]:
def bivariate_plot(df, type, period):
    '''
    Plots the bivariate analysis depending on which procedure was used (mean, max) and the period (week, year)
    Color map makes it easy to distinguish temporal trend (if present)
    Correlation Coefficient is added as a text to show correlation
    '''
    df.plot(kind = 'scatter', x = col[1], y = col[2], c = period, cmap = 'jet',
            title = 'Bivariate Analysis using the ' + type + ' over each ' + period);
    rho = pearson_correlation(df[col[1]], df[col[2]])
    plt.text(500, 80000, f'rho={rho:.3f}');
    plt.show()

In [ ]:
yearly_maxima = pd.DataFrame({col[1]: yearly_maxima_dis[col[1]].values, col[2]: yearly_maxima_sed[col[2]].values,
                              'year': np.arange(1949, 1985)})
yearly_maxima.head()

In [ ]:
bivariate_plot(yearly_maxima, 'max', 'year')

In [ ]:
def weekly_max(data, column):
    '''
    Function that calculates the weekly maximum for a data frame for a given column index
    '''
    col = data.columns.values
    id_max = data.groupby(pd.DatetimeIndex(data[col[0]]).week)[col[column]].idxmax()
    max_list = data.loc[id_max]
    return max_list

In [ ]:
def weekly_mean(data, column):
    '''
    Function that calculates the weekly mean for a data frame for a given column index
    '''
    col = data.columns.values
    list_mean = data.groupby(pd.DatetimeIndex(data[col[0]]).week)[col[column]].mean()
    return list_mean

In [ ]:
dis_week_max = weekly_max(df, 1)
sed_week_max = weekly_max(df, 2)
week_max = pd.DataFrame({col[1]: dis_week_max[col[1]].values, col[2]: sed_week_max[col[2]].values, 'week': np.arange(1, 54)})
week_max.head()

In [ ]:
bivariate_plot(week_max, 'max', 'week')

In [ ]:
dis_week_mean = weekly_mean(df, 1)
sed_week_mean = weekly_mean(df, 2)
week_mean = pd.DataFrame({col[1]: dis_week_mean.values, col[2]: sed_week_mean, 'week': np.arange(1, 54)})
week_mean.head()

In [ ]:
bivariate_plot(week_mean, 'mean', 'week')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ce1ac4c0-44bb-4e75-a664-549207195a6b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>